# EDA: тексты запросов по редиректам

In [16]:
# тексты запросов, где только цифры, с цифрами, короткие, длинные запросы (отдельно по редиректам)
# - количество слов в запросе, бренды (отдельно по редиректам)
# - как часто полностью совпадает с названием категорий (по расс левенштейна/напрямую)

In [28]:
with open("420_redirects.txt") as f:
    searches = [line.strip() for line in f]
searchesLength = len(searches)

In [29]:
queryLengths = {}
queryWordQuantities = {}

for i in range(searchesLength):
    lwords = len(searches[i].split())
    if not (lwords in queryWordQuantities):
        queryWordQuantities[lwords] = 1
    else:
        queryWordQuantities[lwords] += 1
    l = len(searches[i])
    if not (l in queryLengths):
        queryLengths[l] = 1
    else:
        queryLengths[l] += 1
        
queryWordQuantities = {k: v for k, v in sorted(queryWordQuantities.items(), key = lambda item: item[0])}
queryLengths = {k: v for k, v in sorted(queryLengths.items(), key = lambda item: item[0])}

In [30]:
from math import log

sum = 0
print("Количество символов в запросе:")
for k, v in queryLengths.items():
    sum += k * v
    print("{0:3} : {1:4} : ".format(k, v), "*" * int(log(v, 1.2)))
print("\nВ среднем: {0}".format(sum // searchesLength))

Количество символов в запросе:
  2 :    3 :  ******
  3 :   20 :  ****************
  4 :   49 :  *********************
  5 :   67 :  ***********************
  6 :  117 :  **************************
  7 :   98 :  *************************
  8 :  109 :  *************************
  9 :   96 :  *************************
 10 :   91 :  ************************
 11 :   73 :  ***********************
 12 :   83 :  ************************
 13 :   75 :  ***********************
 14 :   59 :  **********************
 15 :   52 :  *********************
 16 :   35 :  *******************
 17 :   34 :  *******************
 18 :   60 :  **********************
 19 :   37 :  *******************
 20 :   30 :  ******************
 21 :   31 :  ******************
 22 :   20 :  ****************
 23 :   26 :  *****************
 24 :   13 :  **************
 25 :   17 :  ***************
 26 :    8 :  ***********
 27 :   14 :  **************
 28 :   13 :  **************
 29 :   11 :  *************
 30 :   12 :  **

In [31]:
print("Количество слов в запросе:")
for k, v in queryWordQuantities.items():
    print("{0:3} : {1:5} : ".format(k, v), "*" * int(log(v, 1.2)))

Количество слов в запросе:
  1 :   518 :  **********************************
  2 :   459 :  *********************************
  3 :   211 :  *****************************
  4 :   132 :  **************************
  5 :    55 :  *********************
  6 :    13 :  **************
  7 :     9 :  ************
  8 :     5 :  ********
  9 :     3 :  ******
 10 :     4 :  *******
 11 :     1 :  
 12 :     1 :  
 13 :     1 :  


In [32]:
import re

brands = []
with open("brands.txt") as raw:
    for row in raw:
        brands.append(re.sub("[^а-яА-Яa-zA-Z0-9]", "", row).lower())
brands = sorted(list(set(brands)))

In [33]:
categories = []
with open("categories.txt") as raw:
    for row in raw:
        categories.append(re.sub("[^а-яА-Яa-zA-Z0-9]", "", row).lower())
categories = sorted(list(set(categories)))

In [34]:
#%pip install python-Levenshtein
from Levenshtein import distance as lev

In [35]:
errorful = []
numerical = 0
containingNumbers = []
containingBrands = []
onlyBrands = 0
equalToCategory = 0
similarToCategory = []
for i in range(searchesLength):
    q = searches[i]
    qStripped = re.sub("[^а-яА-Яa-zA-Z0-9]", "", q)
    if (not qStripped):
        errorful.append(q)
    elif (qStripped == re.sub("[^0-9]", "", qStripped)):
        numerical += 1
    else:
        if (qStripped != re.sub("[^а-яА-Яa-zA-Z]", "", qStripped)):
            containingNumbers.append(q)
        qLower = q.lower()
       
        containsBrand = False
        for brand in brands:
            if (qLower.find(brand) != -1):
                containsBrand = True
                if (qLower == brand):
                    onlyBrands += 1
                break
        if (containsBrand):
            containingBrands.append(q)
        
        mivLev = len(qLower)
        for category in categories:
            if (lev(qLower, category) < len(qLower) // 3):
                similarToCategory.append(q)
                if (qLower == category):
                    equalToCategory += 1
                    break

In [36]:
def percent(val):
    return val * 100 // searchesLength

print("Запросов, содержащих цифры -- {0} ({1}%). Из них состоящих из цифр -- {2} ({3}%)."
      .format(len(containingNumbers), percent(len(containingNumbers)), 
              numerical, percent(numerical)))
print("Запросов, содержащих названия бренда -- {0} ({1}%). Из них состоящих только из названия -- {2} ({3}%)."
      .format(len(containingBrands), percent(len(containingBrands)),
             onlyBrands, percent(onlyBrands)))
print("Запросов, совпадающих с названием категории -- {0} ({1}%)."
      .format(equalToCategory, percent(equalToCategory)))
print("Запросов, похожих на название категории (с малым расстоянием Левентшейна) -- {0} ({1}%)."
      .format(len(similarToCategory), percent(len(similarToCategory))))

Запросов, содержащих цифры -- 32 (2%). Из них состоящих из цифр -- 1 (0%).
Запросов, содержащих названия бренда -- 1064 (75%). Из них состоящих только из названия -- 416 (29%).
Запросов, совпадающих с названием категории -- 428 (30%).
Запросов, похожих на название категории (с малым расстоянием Левентшейна) -- 795 (56%).


In [37]:
print(errorful)

[]


In [38]:
print(similarToCategory[0:20])

['Binturong', 'Blend Smart', 'BlendSMART', 'BIO World', 'Bio-Oil', 'парфюмерная вода dior', 'Dior', 'dior парфюмерная вода', 'sultane de saba', 'La Sultane de Saba', 'Kevin Murphy', 'Kevin.Murphy', 'Kevin Murphey', 'dior backstage eye', 'Dior Backstage', 'Biocos', 'joy by dior', 'miss dior', 'тональная основа dior', 'dior forever skin correct']
